# The Essentials Function for Sub Space

In [15]:
using QuantumOptics

In [16]:
"""
Compute the single-particle operator from single-particle matrix.

#### Arguments
- `sp_basis::NLevelBasis`: Single-Particle basis.
- `N::Integer`: The total site number.
- `sp_matrix::Matrix`: Hopping phases matrix from any model.
"""
function get_sp_op(sp_basis, sp_matrix)
    
    H = SparseOperator(sp_basis)

    N, = size(sp_matrix)
    
    for m in 1:N
        for n in 1:N
            H += sp_matrix[m,n] * transition(sp_basis, m, n)
        end
    end
    
    return H
end

get_sp_op

In [3]:
"""
Compute the first eigen-states in a given number.

#### Arguments
- `sp_op::get_sp_op`: Single-particle operator.
- `cut_off::Int`: Limit value of eigen-state index.
"""
function get_sub_states(sp_op, cut_off)
    
    E0, states0 = eigenstates(dense(sp_op))
    states = states0[1:cut_off]
    
    return states
end

get_sub_states

In [18]:
"""
Compute sub-space basis, projection and comlex conjugate of projection operator.

#### Arguments
- `states::get_sub_states`: Eigen-states of the sub-space.
- `basis::NLevelBasis`: Single-particle basis.
"""
function get_projector_op(states, sp_basis)
    
    sub_basis = SubspaceBasis(sp_basis,states)
    P = projector(sub_basis, sp_basis)
    Pt = dagger(P)
    
    return sub_basis, P, Pt
end

get_projector_op

<font size="5">
$\hat{O}_{sub}=P\hat{O}P^\dagger$
</font>

In [19]:
"""
Compute the corresponding operator in the sub-space.

#### Arguments
- `sp_op::Operator`: Single-particle operator from single-particle matrix.
- `P::get_projector_op[2]`: Projection operator.
- `Pt::get_projector_op[3]`: Complex conjugate of projection operator.
"""
function get_subspace_op(sp_op, P, Pt)
    return P*sp_op*Pt
end

get_subspace_op

<font size="5">
    $n_i=a_i^\dagger a_i$
</font>

In [20]:
"""
Compute the single-particle number operator for each lattice sites.

#### Arguments
- `N::Integer`: The total site number.
- `sp_basis::NLevelBasis`: Single-Particle basis.
- `P::get_projector_op[2]`: Projection operator.
- `Pt::get_projector_op[3]`: Complex conjugate of projection operator.
"""
function get_num_sub_list(N, sp_basis, P, Pt)
    num_sub_list = []
    for m in 1:N
        num_op = transition(sp_basis, m, m)
        num_sub_op = get_subspace_op(num_op, P, Pt)
        push!(num_sub_list, num_sub_op)
    end
    return num_sub_list
end

get_num_sub_list

<font size="5">
    $\hat{O}=\sum_{ij} a^\dagger_i a_j <u_i|\hat{o}|u_j>$
</font>

In [21]:
"""
Compute the many-body operator for boson particles from single-particle operator.

#### Arguments
- `mb_basis`: Many-body basis.
- `sp_op::Operator`: Single-particle operator.
"""
function get_mb_op(mb_basis, sp_op)
    
    mb_op = SparseOperator(mb_basis)
    
    N = sp_op.basis_l.N
    
    for i in 1:N
        for j in 1:N
            mb_op += sp_op.data[i,j] * transition(mb_basis, i, j)
        end
    end
    
    return mb_op
end

get_boson_mb_op

In [ ]:
using Einsum

<font size="5">
    $\hat{V}=\sum_{ijkl}a^\dagger_ia^\dagger_ja_ka_l <u_i|<u_j|\hat{v}|u_k>|u_l>$
</font>

In [13]:
function Hubbard_Interaction(P, Pt, cut_mb_basis, cut_off, U)
    
    P1 = P.data
    P1t = Pt.data

    @einsum coefficient[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    Vint_mb_cut = SparseOperator(cut_mb_basis)
        
    for k in 1:cut_off
        for l in 1:cut_off
            for m in 1:cut_off
                for n in 1:cut_off
                    a1t = create(cut_mb_basis, k)
                    a2t = create(cut_mb_basis, l)
                    a2  = destroy(cut_mb_basis, m)      
                    a1  = destroy(cut_mb_basis, n)      
                    Vint_mb_cut += U/2*coefficient[k,l,m,n]*a1t*a2t*a2*a1
                end
            end
        end
    end
    
    return Vint_mb_cut
end

Hubbard_Interaction

In [ ]:
function get_num_mb_op(N, cut_sp_basis, num_sub_list, cut_mb_basis, sub_basis)
    
    num_sp_op_list = []
    for i in 1:N
        number_sp_list = Operator(cut_sp_basis, num_sub_list[i].data)
        push!(num_sp_op_list, number_sp_list)
    end
    
    num_mb_op_list = []
    for i in 1:N
        number_mb_list = get_mb_op(cut_mb_basis, sub_basis, num_sp_op_list[i])
        push!(num_mb_op_list, number_mb_list)
    end
    
    return num_mb_op_list
end